Ensure you have Selected PySpark(Python) as the language of choice for this notebook in the Ribbon: 
<add image here>

In [1]:
%pip install -U fabric-data-agent-sdk

StatementMeta(, 4176f436-8692-4bea-88ea-dfff7327c2ef, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of azure-kusto-data to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.2 MB/s eta 0:00:00
Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 15.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ✅ TASK OVERVIEW

**🎯 Goal**  
Configure your **Data Agent Artifact** using AI instructions, data source setup, and few-shot examples — aiming for **100% correctness** on all questions for the Dataset of your choice.

## 📝 Pre-Requisite
**Ensure you have completed Step 1 in the Lab 3 Curate Your Data Agent Document**
The DataAgent created in Step 1 of the "Lab 3 Curate Your Data Agent" Document will be the DataAgent used for this Notebook.

---

## 🧪 STEP 1: Use the Evaluation Notebook

1. **Update the `YOUR_DATA_AGENT_ARTIFACT_NAME`**  
   - Replace the placeholder with the name of your Data Agent Artifact from in Step 1 of the "Lab 3 Curate Your Data Agent" Document. 
2. **Update the `YOUR_NAME` Variable**
   - Replace the placeholder with your name

In [3]:
from fabric.dataagent.client import FabricDataAgentManagement
from fabric.dataagent.evaluation import evaluate_data_agent, get_evaluation_summary, get_evaluation_details, get_evaluation_summary_per_question
import openai
import os
import re

YOUR_DATA_AGENT_ARTIFACT_NAME = "KaggleAgent"

YOUR_NAME = "Shreyas"

os.environ["OPENAI_API_VERSION"] = "2023-05-15"

data_agent = FabricDataAgentManagement(YOUR_DATA_AGENT_ARTIFACT_NAME)

StatementMeta(, 4176f436-8692-4bea-88ea-dfff7327c2ef, 10, Finished, Available, Finished)

In [4]:
SELECTABLE_ELEMENT_TYPES = [
    "semantic_model.table",
    "lakehouse_tables.table",
    "warehouse_tables.table",
    "kusto.table",
]

tables_list = []

res = data_agent.get_datasources()[0].get_configuration()
res.get('elements')

selected_dbs=[]

def is_child_selected(elements) -> bool:
    if(elements is None or len(elements) == 0):
        return False

    res = False
    for elem in elements:
        if elem.get('type') in SELECTABLE_ELEMENT_TYPES and elem.get("is_selected"):
            tables_list.append(elem.get("display_name"))
            res = True
    return res
    return is_child_selected(elem.get('children'))

for element in res.get('elements'):
    if(is_child_selected(element.get('children'))):
        selected_dbs.append(element.get('display_name'))

selected_dbs

selectedListStr = '(' + ', '.join(f'"{item}"' for item in selected_dbs) + ')'
print(selectedListStr)


StatementMeta(, 4176f436-8692-4bea-88ea-dfff7327c2ef, 11, Finished, Available, Finished)

("TheHistoryofBaseball")


## 🔄 STEP 2: Understanding the Groundtruth

This step takes the **groundtruths** and displays the **expected answers** and their associated **queries**.

💡 This is the reference we will use to evaluate your Data Agent's performance.

In [5]:
query = "SELECT * FROM Kaggle.groundtruth.groundtruth_expanded WHERE benchmark IN " + selectedListStr + " LIMIT 1000"
query_all_data = "SELECT * FROM Kaggle.groundtruth.groundtruth_expanded LIMIT 1000"
print(query)
groundtruth_df = spark.sql("SELECT * FROM Kaggle.groundtruth.groundtruth_expanded WHERE benchmark IN" + selectedListStr + "LIMIT 1000")
#groundtruth_df = spark.sql("SELECT * FROM Kaggle.groundtruth.groundtruth_expanded")


df_pandas_groundtruth = groundtruth_df.toPandas()
# df_pandas_groundtruth.rename(columns={'sql': 'query'}, inplace=True)
type(df_pandas_groundtruth)

#df_pandas_groundtruth['query'] = df_pandas_groundtruth.apply(lambda row: qualify_sql(sql=row['sql'], schema=row['benchmark'], table_list=tables_list), axis=1)
#df_pandas_groundtruth.rename(columns={'answer': 'expected_answer'}, inplace=True)

display(df_pandas_groundtruth)

StatementMeta(, 4176f436-8692-4bea-88ea-dfff7327c2ef, 12, Finished, Available, Finished)

SELECT * FROM Kaggle.groundtruth.groundtruth_expanded WHERE benchmark IN ("TheHistoryofBaseball") LIMIT 1000


SynapseWidget(Synapse.DataFrame, 60a64637-4aff-4cf6-abb2-52e8f0b6edf1)

## 📊 STEP 3: Evaluation

This is the **actual evaluation step**, where a new **conversation thread** is started with the Data Agent for **each question**.

- We use an **LLM-critic approach** to assess the agent’s answer against the expected answer.  
  > 🧠 *LLM-critic*: A language model is used to **judge** whether the generated response matches the expected one — even when the phrasing differs.
  
- ✅ **Note**: We **do not compare** the SQL queries. Only the **final answer text** is evaluated.

---

### 📄 Evaluation Output Columns

| Column           | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| **T**            | ✅ Number of correct answers (matches expected output)                      |
| **F**            | ❌ Number of incorrect answers                                               |
| **?**            | 🤔 Unclear — the LLM critic couldn’t decide                                 |
| **%**            | ✔️ Percentage of T out of total (T + F + ?)                                 |
| **failed thread**| 🔗 Links to failed threads (for manual inspection)                          |
| **question**     | 📝 The question from the ground truth                                       |

> 🔁 You can use `num_query_repeat` to ask the same question multiple times to assess variability in responses.

---

### ✅ Next Steps

- Publish your DataAgent
- Inspect the threads that **have mismatches**
- Update:
  - AI instructions
  - Data source instructions
  - Few-shot examples  
- **Re-run the evaluation**


In [6]:

# The table used to store evaluation results
evaluation_table_name = f'evaluation_output_{YOUR_NAME}'

# run evaluation

eval_id = evaluate_data_agent(
    df = df_pandas_groundtruth,
    data_agent_name = YOUR_DATA_AGENT_ARTIFACT_NAME,
    data_agent_stage = "production",
    table_name = evaluation_table_name)

df = get_evaluation_summary_per_question(eval_id, verbose=True, table_name = evaluation_table_name)

StatementMeta(, 4176f436-8692-4bea-88ea-dfff7327c2ef, 13, Finished, Available, Finished)

**🔗Failed Thread(s):**
- [thread_2vkvTzJ9ZFyR1uFdXDeJXjLP](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_2vkvTzJ9ZFyR1uFdXDeJXjLP&debug.aiSkillViewPublishedOverride=0)
- [thread_v7nJqkICJSq0LQVbaJw9tPGO](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_v7nJqkICJSq0LQVbaJw9tPGO&debug.aiSkillViewPublishedOverride=0)
- [thread_GiXjTgG4i8a3VCmGX9DVDJ9c](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_GiXjTgG4i8a3VCmGX9DVDJ9c&debug.aiSkillViewPublishedOverride=0)
- [thread_AII3nMpsZfqHjtjQDzxwAEap](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_AII3nMpsZfqHjtjQDzxwAEap&debug.aiSkillViewPublishedOverride=0)
- [thread_iKnG1RwtgUNk0YEaNmQi8qAc](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_iKnG1RwtgUNk0YEaNmQi8qAc&debug.aiSkillViewPublishedOverride=0)
- [thread_ZO8VvoDkT1RxjuB6bVyX9bTs](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_ZO8VvoDkT1RxjuB6bVyX9bTs&debug.aiSkillViewPublishedOverride=0)
- [thread_VO4PP3WKbUuAtDtCiy2UxlmS](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_VO4PP3WKbUuAtDtCiy2UxlmS&debug.aiSkillViewPublishedOverride=0)
- [thread_nFtGard2XihbT9FhEdwrnBVV](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_nFtGard2XihbT9FhEdwrnBVV&debug.aiSkillViewPublishedOverride=0)
- [thread_SYB2tn93Tr8xBHBnRPCoPJKM](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_SYB2tn93Tr8xBHBnRPCoPJKM&debug.aiSkillViewPublishedOverride=0)
- [thread_OzQkyf9oX7W1ZwYDzYsW0P5h](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_OzQkyf9oX7W1ZwYDzYsW0P5h&debug.aiSkillViewPublishedOverride=0)
- [thread_ZPlvVM8cEGHlAaWEmIkb7QX6](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_ZPlvVM8cEGHlAaWEmIkb7QX6&debug.aiSkillViewPublishedOverride=0)
- [thread_aMl07sxuI7t3EAOIwnDNGRW9](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_aMl07sxuI7t3EAOIwnDNGRW9&debug.aiSkillViewPublishedOverride=0)
- [thread_x5fkz9GSU0fvQh8WLtNmLMPm](https://dxt.fabric.microsoft.com/workloads/de-ds/dataagents/92ff74dc-64d4-45f5-b5a9-3cf627d24f5e/externalThread?debug.aiSkillThreadIdOverride=thread_x5fkz9GSU0fvQh8WLtNmLMPm&debug.aiSkillViewPublishedOverride=0)

100%|██████████| 38/38 [02:08<00:00,  3.37s/it]


index,T,F,?,%,failed_threads,question
0,0,1,0,0.0,1,Average lifespan of hall of fame players
1,0,0,1,0.0,1,"For award winners, what's average weight for each position"
2,1,0,0,100.0,No failed records,"For award winners, which is the position that has the most hall of fame players?"
3,0,1,0,0.0,1,"For the All-Star Game MVP, who is the youngest winner?"
4,1,0,0,100.0,No failed records,How many players weigh over 200 pounds?
5,1,0,0,100.0,No failed records,How many players were awarded more than ten times?
6,1,0,0,100.0,No failed records,How many points were received by the NL league's most valuable player winner in 1990?
7,1,0,0,100.0,No failed records,In what defense position is the player ID willite01?
8,0,1,0,0.0,1,Top 10 countries with the most hall of fame players
9,1,0,0,100.0,No failed records,Top 10 teams with the most hall of fame players
